# Mechanisms of Action (MoA) Prediction

In [2]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, UpSampling2D, LeakyReLU, Concatenate,Dense,Input,Flatten,BatchNormalization,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras import Model

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from random import randint


In [3]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')
train_targets_scored = pd.read_csv('train_targets_scored.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [4]:
train_features.loc[(train_features['cp_type'] == "trt_cp"), 'cp_type'] = 0
train_features.loc[(train_features['cp_type'] == "ctl_vehicle"), 'cp_type'] = 1

train_features.loc[(train_features['cp_dose'] == "D1"), 'cp_dose'] = 0
train_features.loc[(train_features['cp_dose'] == "D2"), 'cp_dose'] = 1

train_features[["cp_dose", "cp_type"]] = train_features[["cp_dose", "cp_type"]].apply(pd.to_numeric)

test_features.loc[(test_features['cp_type'] == "trt_cp"), 'cp_type'] = 0
test_features.loc[(test_features['cp_type'] == "ctl_vehicle"), 'cp_type'] = 1

test_features.loc[(test_features['cp_dose'] == "D1"), 'cp_dose'] = 0
test_features.loc[(test_features['cp_dose'] == "D2"), 'cp_dose'] = 1

test_features[["cp_dose", "cp_type"]] = test_features[["cp_dose", "cp_type"]].apply(pd.to_numeric)

In [5]:
feature_variables = list(set([i for i in test_features.columns])-set(['cp_type','sig_id']))
target_variables =  list(set([i for i in sample_submission.columns])-set(['sig_id']))

In [6]:
Dataset = pd.merge(train_features,train_targets_scored,on='sig_id')
print(Dataset.shape)
Dataset.drop(Dataset[Dataset['cp_type']==1].index,inplace=True)
print(Dataset.shape)

(23814, 1082)
(21948, 1082)


In [7]:
Test_Dataset = pd.merge(test_features,sample_submission,on='sig_id')
Test_Dataset.loc[Test_Dataset['cp_type']==1,target_variables]=0

In [8]:
x_train = Dataset[feature_variables].to_numpy()
y_train = Dataset[target_variables].to_numpy()

In [9]:
input_feature_shape=len(feature_variables)
output_feature_shape=len(target_variables)

model_input = Input(shape = input_feature_shape)

x = Dense(units = 600,activation="relu")(model_input)
x = BatchNormalization()(x)
x = Dropout(rate = 0.1737384812871308)(x)

x = Dense(units = 298,activation="sigmoid")(model_input)
x = BatchNormalization()(x)
x = Dropout(rate = 0.20010328415413295)(x)

x = Dense(units = 1099,activation="sigmoid")(model_input)
x = BatchNormalization()(x)
x = Dropout(rate = 0.3234867319850344)(x)


Output = Dense(output_feature_shape,activation='sigmoid')(x)

#define model
model = Model(model_input,Output,name = 'BaseLine')

2022-09-03 16:23:19.132161: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
factor= 0.8250037987063858
patience=2
min_lr= 5.101088055532695e-05
lr=6.353131263848553e-05
batch_size=353
epochs=150


file_path = model.name+'best_weights.hd5'

optimizer = tf.keras.optimizers.Adam(lr=lr, amsgrad=True)
#compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=optimizer)

history=model.fit(x_train,y_train,epochs= epochs, batch_size=batch_size)

/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/150
63/63 [==============================] - 3s 35ms/step - loss: 0.8549
Epoch 2/150
63/63 [==============================] - 2s 39ms/step - loss: 0.8101
Epoch 3/150
63/63 [==============================] - 2s 39ms/step - loss: 0.7857
Epoch 4/150
63/63 [==============================] - 3s 41ms/step - loss: 0.7600
Epoch 5/150
63/63 [==============================] - 3s 46ms/step - loss: 0.7286
Epoch 6/150
63/63 [==============================] - 3s 42ms/step - loss: 0.6880
Epoch 7/150
63/63 [==============================] - 3s 47ms/step - loss: 0.6334
Epoch 8/150
63/63 [==============================] - 2s 39ms/step - loss: 0.5685
Epoch 9/150
63/63 [==============================] - 4s 69ms/step - loss: 0.4961
Epoch 10/150
63/63 [==============================] - 3s 47ms/step - loss: 0.4222
Epoch 11/150
63/63 [==============================] - 3s 47ms/step - loss: 0.3530
Epoch 12/150
63/63 [==============================] - 3s 50ms/step - loss: 0.2891
Epoch 13/150
63/63 [=====

63/63 [==============================] - 5s 80ms/step - loss: 0.0183
Epoch 101/150
63/63 [==============================] - 5s 85ms/step - loss: 0.0183
Epoch 102/150
63/63 [==============================] - 3s 52ms/step - loss: 0.0183
Epoch 103/150
63/63 [==============================] - 3s 54ms/step - loss: 0.0182
Epoch 104/150
63/63 [==============================] - 4s 63ms/step - loss: 0.0181
Epoch 105/150
63/63 [==============================] - 3s 52ms/step - loss: 0.0181
Epoch 106/150
63/63 [==============================] - 3s 50ms/step - loss: 0.0180
Epoch 107/150
63/63 [==============================] - 4s 60ms/step - loss: 0.0180
Epoch 108/150
63/63 [==============================] - 4s 62ms/step - loss: 0.0179
Epoch 109/150
63/63 [==============================] - 5s 79ms/step - loss: 0.0179
Epoch 110/150
63/63 [==============================] - 3s 46ms/step - loss: 0.0179
Epoch 111/150
63/63 [==============================] - 4s 64ms/step - loss: 0.0179
Epoch 112/150
63/6

In [12]:
y_pred=model.predict(Test_Dataset[feature_variables])
Test_Dataset[target_variables] = y_pred
Test_Dataset.loc[Test_Dataset['cp_type']==1,target_variables]=0

125/125 [==============================] - 1s 4ms/step
